In [76]:
import logging
import pandas as pd
import azure.functions as func
import logging
from azure.storage.blob import BlobServiceClient
import os
import pyodbc
import re

In [2]:
incoming_input_pdf_nbd = "pdf/processed/Account_Details_03Jun2023_133827_Emirates-NBD-Classic-Luxury-Main_2023-06-14T17:05:30.3232782Z.pdf"
incoming_input_pdf_cbd = "pdf/processed/Account_Details_03Jun2023_133827_CBD_2023-06-14T17:05:30.3232782Z.pdf"
incoming_input_pdf_cltrak = "pdf/processed/Account_Details_03Jun2023_133827_CLTRAK_2023-06-14T17:05:30.3232782Z.pdf"

In [78]:
template_file_path = "https://arunakcs.blob.core.windows.net/excelfiles/main_template/test_template.xlsx"
df_template = pd.read_excel(template_file_path)
df_reference = pd.read_excel(template_file_path, sheet_name="term_references")
server = "akcserver.database.windows.net"
database = "dbarunsql"
username = "Arun"
password = "Asds@2022"
table_name = "TransactionDetails"

In [111]:
try:
    # Establish the database connection
    conn_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    conn = pyodbc.connect(conn_str)

    # Read rows from the SQL table
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {table_name} where TransactionId=13870")
    rows = cursor.fetchall()

    # Get column names from the cursor description
    column_names = [column[0] for column in cursor.description]

    # Convert rows to a list of dictionaries
    data = []
    for row in rows:
        data.append(dict(zip(column_names, row)))

    # Create DataFrame from the data
    df_input_bank_statement_from_sql = pd.DataFrame(data)

except Exception as e:
    raise

finally:
    conn.close()

In [112]:
from datetime import datetime

In [113]:
def read_user_input_data(input_file, df_input):
    current_datetime = datetime.now()
    Current_Date= current_datetime.strftime("%Y-%m-%d")
    filter_date = pd.to_datetime(Current_Date).date()
    print(filter_date)
    df_input['ModelDate'] = pd.to_datetime(df_input['ModelCopyDateTime'])
    df_input['ModelDateExtracted']=df_input['ModelDate'].dt.date
    df_input =  df_input[df_input['ModelDateExtracted']==filter_date]
    if 'NBD' in input_file:
        nbd_df = df_input[df_input['DomainName']=='NBD']  
        return nbd_df
    
    elif 'CLTRAK' in input_file:
        clt_rak_df = df_input[df_input['DomainName']=='CLTRAK']
        return clt_rak_df
    
    elif 'CBD' in input_file:
        cbd_df = df_input[df_input['DomainName']=='CBD']
        return cbd_df
    
    else: 
        print(f'{input_file} Does not exist.')

In [114]:
out = read_user_input_data(incoming_input_pdf_nbd, df_input_bank_statement_from_sql)

2023-06-15


In [115]:
out['TransactionDate'].isnull().sum()

0

In [116]:
def general_preprocess(input_df):
    dropped_df = input_df['TransactionDate'].dropna()
    return input_df

def preprocess_text_data(input_df):
    lowered_narration_data_series = input_df['Narration'].apply(lambda x: str(x).lower() if x is not None else None)
    input_df['lowered_narration'] = lowered_narration_data_series
    return input_df

In [117]:
a = general_preprocess(out)


In [118]:
new_nbd = preprocess_text_data(a)

In [119]:
def preprocess_template_data(input_dataframe):
    lowered_description = input_dataframe['Description'].apply(str.lower)
    input_dataframe['description_lowered'] = lowered_description
    return input_dataframe

In [87]:
new_template_df = preprocess_template_data(df_template)

In [88]:
df_reference

,Terminology,References
0,Cheques Deposited,.outward t/t.*
1,Inward Remittance,.*inward remittance.*
2,Bounced / Returned cheque/DDR,NaN
3,Cash Deposited,.*sdm deposit.*
4,Internal transfer,inward remittance|inter group
5,Credit card payment,.*credit card.*
6,Cheques issued,NaN
7,Bank Charges,^(?!.*visa charges).*charges.*value added
8,Cash Withdrawn,NaN
9,Card Credits,.*network international.*|.*amex.*|.*fds-tos.*


In [120]:
new_nbd = new_nbd.dropna(subset=['Narration'])
new_nbd_without_nulls = new_nbd.dropna(subset=['Narration'])


In [121]:
new_transactions = new_nbd_without_nulls['Narration']
print(len(new_transactions))

1


In [122]:
new_transactions

0    INWARD REMITTANCETT REF: AEL2306130005732 AED ...
Name: Narration, dtype: object

In [123]:
new_nbd

,TransactionId,TransactionDate,Narration,Debit,Credit,RunningBalance,ModelCopyDateTime,file_name,DomainName,CreatedDate,ModifiedDate,Classified,LabelTerm,ModelDate,ModelDateExtracted,lowered_narration
0,13870,14-06-2023,INWARD REMITTANCETT REF: AEL2306130005732 AED ...,0.0,19662.3,650092.9,2023-06-15 13:13:53.563,OLT - EIB.pdf,NBD,2023-06-15 13:13:53.563,2023-06-15 13:13:53.563,No,None,2023-06-15 13:13:53.563,2023-06-15,inward remittancett ref: ael2306130005732 aed ...


In [124]:
from joblib import dump, load
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# Load the saved model
model = load('AkcsNlpCustommodel.pkl')

# Load the saved vectorizer
vectorizer = load('vectorizer.pkl')

# New data
new_transactions = new_nbd_without_nulls['Narration']


# Vectorize the new data
new_transactions_vectorized = vectorizer.transform(new_transactions)

# Predict using the loaded model
predictions = model.predict(new_transactions_vectorized)

# Print the predictions
for transaction, prediction in zip(new_transactions, predictions):
    print(f'Transaction: {transaction}\nPrediction: {prediction}')

new_nbd['Prediction'] = predictions

Transaction: INWARD REMITTANCETT REF: AEL2306130005732 AED 19662.3 THE BRITISH C OUNCIL
/REF/AEL2306130005732 10207 50722 PA
Prediction: Unclassified


In [125]:
new_nbd['Prediction'].value_counts()

Unclassified    1
Name: Prediction, dtype: int64

In [103]:
new_nbd[new_nbd['Prediction'] == 'Bank Charges']['Debit'].sum()

0.0

In [104]:
new_nbd[new_nbd['Prediction'] == 'Cash Deposited']['Credit'].sum()

12700.0

In [68]:
df_template

,Description,Emirates NBD-Classic Luxury-Main,Rak Bank-Classic Luxury,CLT - ADCB,CBD Bank,EIB - Loan account,OLT - Emirates Islamic Bank,Emirates NBD-Classic Passenger,ENBD - Classic Riders,description_lowered
0,Opening Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,opening balance
1,Cheques Deposited,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cheques deposited
2,Inward Remittance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inward remittance
3,Bounced / Returned cheque/DDR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bounced / returned cheque/ddr
4,Cash Deposited,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cash deposited
5,Internal transfer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,internal transfer
6,Credit card payment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,credit card payment
7,Cheques issued,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cheques issued
8,Bank Charges,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bank charges
9,Cash Withdrawn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cash withdrawn


In [105]:
df_template.loc[df_template['Description'] == 'Inward Remittance', 'Emirates NBD-Classic Luxury-Main']

2    92152.2
Name: Emirates NBD-Classic Luxury-Main, dtype: float64

In [106]:
new_descriptions  = df_template['Description'].unique()

In [107]:
def populate_final_report(report_template, nlp_classified_df, input_file_path):
    if 'NBD' in input_file_path:
        
        for description in report_template['Description']:
            filtered_df = nlp_classified_df[nlp_classified_df['Prediction'] == description]
            debit_sum = filtered_df['Debit'].sum()
            credit_sum = filtered_df['Credit'].sum()
            total_sum = credit_sum + (-debit_sum)
            report_template.loc[report_template['Description'] == description, 'Emirates NBD-Classic Luxury-Main'] = total_sum
        report_template.loc[report_template['Description'] == 'Closing Balance at the day end', 'Emirates NBD-Classic Luxury-Main'] = report_template['Emirates NBD-Classic Luxury-Main'][1:12].sum()
        
        return report_template

In [108]:
latest_report = populate_final_report(df_template, new_nbd, incoming_input_pdf_nbd)

In [73]:

# # Iterate over each description in the new DataFrame
# for description in df_template['Description']:
#     # Filter previous DataFrame based on matching prediction
#     filtered_df = new_nbd[new_nbd['Prediction'] == description]
    
#     # Calculate the sum of debit and credit values
#     debit_sum = filtered_df['Debit'].sum()
#     credit_sum = filtered_df['Credit'].sum()
#     total_sum = credit_sum + (-debit_sum)
#     df_template.loc[df_template['Description'] == description, 'Emirates NBD-Classic Luxury-Main'] = total_sum



In [110]:
latest_report.to_excel("report.xlsx")

In [35]:
from azure.storage.blob import BlobServiceClient
import pickle
connection_string = "DefaultEndpointsProtocol=https;AccountName=arunakcs;AccountKey=nx8T5960W1vcaeHKOD/4HtiCm0/n58VXhtsNAp7LoyDdZX6IdRPsomJsBoOgB72wPd9AHfwwcoFo+AStndZq2Q==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client("akcsaiamodel")
blob_client = container_client.get_blob_client("vectorizer.pkl")
blob_data = blob_client.download_blob().readall()


In [37]:
loaded_data = pickle.loads(blob_data)

UnpicklingError: invalid load key, '\x0a'.

In [7]:
import logging
import pandas as pd
import azure.functions as func
import logging
from azure.storage.blob import BlobServiceClient, ContentSettings
import os
import pyodbc
import re
from datetime import datetime
from joblib import dump, load
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

def read_user_input_data(input_file, df_input):
    print('starting point')
    current_datetime = datetime.now()
    Current_Date= current_datetime.strftime("%Y-%m-%d")
    filter_date = pd.to_datetime(Current_Date).date()
    
    logging.info(filter_date)
    
    df_input['ModelDate'] = pd.to_datetime(df_input['ModelCopyDateTime'])
    df_input['ModelDateExtracted']=df_input['ModelDate'].dt.date
    df_input =  df_input[df_input['ModelDateExtracted']==filter_date]
    
    if 'Emirates-NBD-Classic-Luxury-Main' in input_file:
        nbd_df = df_input[df_input['DomainName']=='Emirates-NBD-Classic-Luxury-Main']  
        return nbd_df
    
    elif 'Rak-Bank-Classic-Luxury' in input_file:
        clt_rak_df = df_input[df_input['DomainName']=='Rak Bank-Classic Luxury']
        return clt_rak_df
    
    elif 'CLT-ADCB' in input_file:
        clt_adcb_df = df_input[df_input['DomainName']=='CLT - ADCB']
        return clt_adcb_df
    
    elif 'CBD-Bank' in input_file:
        cbd_df = df_input[df_input['DomainName']=='CBD-Bank']
        return cbd_df
    
    elif 'EIB-Loan-account' in input_file:
        cbd_df = df_input[df_input['DomainName']=='EIB - Loan account']
        return cbd_df
    
    elif 'OLT-Emirates-Islamic-Bank' in input_file:
        cbd_df = df_input[df_input['DomainName']=='OLT - Emirates Islamic Bank']
        return cbd_df

    
    elif 'Emirates-NBD-Classic-Passenger' in input_file:
        emirates_nbd_classic_passenger_df= df_input[df_input['DomainName']=='Emirates NBD-Classic Passenger']
        return emirates_nbd_classic_passenger_df
    
    elif 'ENBD-Classic-Riders' in input_file:
        enbd_classic_riders_df = df_input[df_input['DomainName']=='ENBD - Classic Riders']
        return enbd_classic_riders_df
    
    else: 
        print(f'{input_file} Does not exist.')
        
def general_preprocess(input_df):
    if input_df['TransactionDate'].isnull().sum() > 0:
        dropped_df = input_df['TransactionDate'].dropna()
        print(dropped_df)
        return input_df
    return input_df

def preprocess_template_data(input_dataframe):
    lowered_description = input_dataframe['Description'].apply(str.lower)
    input_dataframe['description_lowered'] = lowered_description
    return input_dataframe

def preprocess_text_data(input_df):
    lowered_narration_data_series = input_df['Narration'].apply(lambda x: str(x).lower() if x is not None else None)
    input_df['lowered_narration'] = lowered_narration_data_series
    return input_df

def predict_transactions(new_transactions, model_weights, vectorizer_weights):
    # Load the saved model

    # Vectorize the new data
    new_transactions_vectorized = vectorizer_weights.transform(new_transactions)

    # Predict using the loaded model
    predictions = model_weights.predict(new_transactions_vectorized)

    return predictions

def fetch_data_from_sql(server, database, username, password, table_name):
    # Establish the database connection
    print(server, database, username, password, table_name)
    conn_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    
    conn = pyodbc.connect(conn_str)

    # Read rows from the SQL table
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()

    # Convert the result to a DataFrame
    df =pd.DataFrame([tuple(row) for row in rows], columns=[column[0] for column in cursor.description])
    conn.close()
    return df

def populate_final_report(report_template, nlp_classified_df, input_file_path):
    server = "akcserver.database.windows.net"
    database = "dbarunsql"
    username = "Arun" 
    password = "Asds@2022"
    driver = '{ODBC Driver 17 for SQL Server}'  # Update the driver if needed
    
    conn_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    conn = pyodbc.connect(conn_str)
    closingbalanceTableName = "DailyTransactionBalance"
    CreatedDate = datetime.now()
    ModifiedDate = datetime.now()
    if 'Emirates-NBD-Classic-Luxury-Main' in input_file_path:
        
        for description in report_template['Description']:
            filtered_df = nlp_classified_df[nlp_classified_df['Prediction'] == description]
            debit_sum = filtered_df['Debit'].sum()
            credit_sum = filtered_df['Credit'].sum()
            total_sum = credit_sum + (-debit_sum)
            report_template.loc[report_template['Description'] == description, 'Emirates NBD-Classic Luxury-Main'] = total_sum
        closingBalance = report_template['Emirates NBD-Classic Luxury-Main'][1:12].sum()
        report_template.loc[report_template['Description'] == 'Closing Balance at the day end', 'Emirates NBD-Classic Luxury-Main'] = report_template['Emirates NBD-Classic Luxury-Main'][1:12].sum()
        
        cursor = conn.cursor()
        sql = f"INSERT INTO {closingbalanceTableName} (ClosingBalanceDomainCompany, CreatedDate, ModifiedDate, ClosingBalance) " \
              f"VALUES (?, ?, ?, ?)"

# Prepare the values for the parameters
        params = ('Emirates-NBD-Classic-Luxury-Main', datetime.now(), datetime.now(), closingBalance)

# Execute the SQL statement with the parameters
        cursor.execute(sql, params)
        conn.commit()
        conn.close
        
        return report_template
    elif 'CBD-Bank' in input_file_path:
        
        for description in report_template['Description']:
            filtered_df = nlp_classified_df[nlp_classified_df['Prediction'] == description]
            debit_sum = filtered_df['Debit'].sum()
            credit_sum = filtered_df['Credit'].sum()
            total_sum = credit_sum + (-debit_sum)
            report_template.loc[report_template['Description'] == description, 'CBD Bank'] = total_sum
        report_template.loc[report_template['Description'] == 'Closing Balance at the day end', 'CBD Bank'] = report_template['CBD Bank'][1:12].sum()
        return report_template
    elif 'Rak-Bank-Classic-Luxury' in input_file_path:
        
        for description in report_template['Description']:
            filtered_df = nlp_classified_df[nlp_classified_df['Prediction'] == description]
            debit_sum = filtered_df['Debit'].sum()
            credit_sum = filtered_df['Credit'].sum()
            total_sum = credit_sum + (-debit_sum)
            report_template.loc[report_template['Description'] == description, 'Rak Bank-Classic Luxury'] = total_sum
        report_template.loc[report_template['Description'] == 'Closing Balance at the day end', 'Rak Bank-Classic Luxury'] = report_template['Rak Bank-Classic Luxury'][1:12].sum()
        return report_template
    elif 'CLT-ADCB' in input_file_path:
        
        for description in report_template['Description']:
            filtered_df = nlp_classified_df[nlp_classified_df['Prediction'] == description]
            debit_sum = filtered_df['Debit'].sum()
            credit_sum = filtered_df['Credit'].sum()
            total_sum = credit_sum + (-debit_sum)
            report_template.loc[report_template['Description'] == description, 'CLT-ADCB'] = total_sum
        report_template.loc[report_template['Description'] == 'Closing Balance at the day end', 'CLT-ADCB'] = report_template['CLT-ADCB'][1:12].sum()
        return report_template
    elif 'EIB-Loan-account' in input_file_path:
        
        for description in report_template['Description']:
            filtered_df = nlp_classified_df[nlp_classified_df['Prediction'] == description]
            debit_sum = filtered_df['Debit'].sum()
            credit_sum = filtered_df['Credit'].sum()
            total_sum = credit_sum + (-debit_sum)
            report_template.loc[report_template['Description'] == description, 'EIB-Loan account'] = total_sum
        report_template.loc[report_template['Description'] == 'Closing Balance at the day end', 'EIB-Loan account'] = report_template['EIB-Loan account'][1:12].sum()
        return report_template
    elif 'OLT-Emirates-Islamic-Bank' in input_file_path:
        
        for description in report_template['Description']:
            filtered_df = nlp_classified_df[nlp_classified_df['Prediction'] == description]
            debit_sum = filtered_df['Debit'].sum()
            credit_sum = filtered_df['Credit'].sum()
            total_sum = credit_sum + (-debit_sum)
            report_template.loc[report_template['Description'] == description, 'OLT - Emirates Islamic Bank'] = total_sum
        report_template.loc[report_template['Description'] == 'Closing Balance at the day end', 'OLT - Emirates Islamic Bank'] = report_template['OLT - Emirates Islamic Bank'][1:12].sum()
        return report_template
    elif 'Emirates-NBD-Classic-Passenger' in input_file_path:
        for description in report_template['Description']:
            filtered_df = nlp_classified_df[nlp_classified_df['Prediction'] == description]
            debit_sum = filtered_df['Debit'].sum()
            credit_sum = filtered_df['Credit'].sum()
            total_sum = credit_sum + (-debit_sum)
            report_template.loc[report_template['Description'] == description, 'Emirates-NBD-Classic-Passenger'] = total_sum
        report_template.loc[report_template['Description'] == 'Closing Balance at the day end', 'Emirates-NBD-Classic-Passenger'] = report_template['Emirates-NBD-Classic-Passenger'][1:12].sum()
        return report_template
    elif 'ENBD-Classic-Riders' in input_file:
        for description in report_template['Description']:
            filtered_df = nlp_classified_df[nlp_classified_df['Prediction'] == description]
            debit_sum = filtered_df['Debit'].sum()
            credit_sum = filtered_df['Credit'].sum()
            total_sum = credit_sum + (-debit_sum)
            report_template.loc[report_template['Description'] == description, 'OENBD - Classic Riders'] = total_sum
        closingBalance = report_template['ENBD - Classic Riders'][1:12].sum()
        report_template.loc[report_template['Description'] == 'Closing Balance at the day end', 'ENBD - Classic Riders'] = report_template['ENBD - Classic Riders'][1:12].sum() 
        
        return report_template

from io import BytesIO
def save_dataframe_to_blob(dataframe, connection_string, container_name, excel_file_name):
    # Save DataFrame to Excel file
    report_file = BytesIO()
    dataframe.to_excel(report_file, index=False)
    report_file.seek(0)
    
    
    # Upload Excel file to Blob storage
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(excel_file_name)

    
    blob_client.upload_blob(report_file, overwrite=True, content_settings=ContentSettings(content_type='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))




    
input_file = incoming_input_pdf_nbd
template_file_path = "https://arunakcs.blob.core.windows.net/excelfiles/main_template/test_template.xlsx"
df_template = pd.read_excel(template_file_path)
df_reference = pd.read_excel(template_file_path, sheet_name="term_references")
logging.info(df_template.columns)

server = "akcserver.database.windows.net"
database = "dbarunsql"
username = "Arun" 
password = "Asds@2022"
table_name = "TransactionDetails"
connection_string = "DefaultEndpointsProtocol=https;AccountName=arunakcs;AccountKey=nx8T5960W1vcaeHKOD/4HtiCm0/n58VXhtsNAp7LoyDdZX6IdRPsomJsBoOgB72wPd9AHfwwcoFo+AStndZq2Q==;EndpointSuffix=core.windows.net"    
try:
    print("Trying")
    df_input_bank_statement_from_sql = fetch_data_from_sql(server, database, username, password, table_name)
    print(f"Data has been fetched from {database} and the table {table_name}")
     
    df_input_bank_statement = read_user_input_data(incoming_input_pdf_nbd ,df_input_bank_statement_from_sql)
    print(f"{len(df_input_bank_statement)} Input bank statement")
        
    preprocessed_data = general_preprocess(df_input_bank_statement)
    print(f"{len(preprocessed_data)} Preprocessed data")
        
    pdf_based_file_preprocessed_data = preprocess_text_data(preprocessed_data)
    print(f"{len(pdf_based_file_preprocessed_data)} PDF File preprocessed data")
        
    report_template = preprocess_template_data(df_template)
    print(f"report template")
    
    print('checkpoint 1')    
    nlp_classified = pdf_based_file_preprocessed_data.dropna(subset=['Narration'])
    print('checkpoint 2')
    nlp_classified_data_without_nulls = nlp_classified.dropna(subset=['Narration'])
    print('checkpoint 3')
    nlp_bank_transactions = nlp_classified_data_without_nulls['Narration']
    
    print('checkpoint 4')    
    aiblob_service_client = BlobServiceClient.from_connection_string(connection_string)
    print('checkpoint 5')
    aicontainer_client = aiblob_service_client.get_container_client("akcsaiamodel")
    
 
    print('checkpoint 6')
    aiblob_client_akcsmodel = aicontainer_client.get_blob_client("AkcsNlpCustommodel_V1.pkl")
    print('checkpoint 7')
    aiblob_data_akcsmodel = aiblob_client_akcsmodel.download_blob().readall()
    print('checkpoint 8')
    model_weights = pickle.loads(aiblob_data_akcsmodel)
    
    print('checkpoint 9')
    aiblob_client_vecmodel = aicontainer_client.get_blob_client("Vectorizer_V1.pkl")
    print('checkpoint 10')
    aiblob_data_vecmodel = aiblob_client_vecmodel.download_blob().readall()
    print('checkpoint 11')
    vectorizer_weights = pickle.loads(aiblob_data_vecmodel)
        
    print("Predictiing using Model path")
    predictions = predict_transactions(nlp_bank_transactions, model_weights, vectorizer_weights)
        
        # Print the predictions
    #or transaction, prediction in zip(nlp_bank_transactions, predictions):
     #   print(f'Transaction: {transaction}\nPrediction: {prediction}')

    pdf_based_file_preprocessed_data['Prediction'] = predictions
    populate_report_template = populate_final_report(report_template, pdf_based_file_preprocessed_data, incoming_input_pdf_nbd)
    #print(populate_report_template)
    now_date = datetime.now()
    container_name = "outputreport"
    file_name = f"output_report_{now_date.date()}_{now_date.minute}_{now_date.second}.xlsx"
    print(file_name)    
    save_dataframe_to_blob(populate_report_template,connection_string, container_name, file_name)

except Exception as e:
    logging.error(e)
    


Trying
akcserver.database.windows.net dbarunsql Arun Asds@2022 TransactionDetails
Data has been fetched from dbarunsql and the table TransactionDetails
starting point
22 Input bank statement
22 Preprocessed data
22 PDF File preprocessed data
report template
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 8
checkpoint 9
checkpoint 10
checkpoint 11
Predictiing using Model path
output_report_2023-06-16_39_23.xlsx


In [40]:
container_name = 'functionapp-pdf'
connection_string = "DefaultEndpointsProtocol=https;AccountName=arunakcs;AccountKey=nx8T5960W1vcaeHKOD/4HtiCm0/n58VXhtsNAp7LoyDdZX6IdRPsomJsBoOgB72wPd9AHfwwcoFo+AStndZq2Q==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

blobs = container_client.list_blobs()
latest_blob = max(blobs, key=lambda blob: blob.last_modified)
latest_blob_name = latest_blob.name

In [42]:
from azure.storage.blob import BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
containers = blob_service_client.list_containers()



In [48]:
container_name = '$logs'
container_client = blob_service_client.get_container_client(container_name)

soft_deleted_blobs = container_client.list_blobs(include="deleted")

In [50]:
for blob in soft_deleted_blobs:
    container_client.undelete_blob(blob.name, blob.version_id)


AttributeError: 'ContainerClient' object has no attribute 'undelete_blob'